To discover node information, set container IPs and execute all the cells

In [ ]:
haproxy_ip='172.17.0.7'
httpd_ip='172.17.0.9'

In [ ]:
from fabric.api import *
env.user='root'
env.password='rootpass'

In [ ]:
import os
with open('/tmp/jupyter/haproxy.cfg') as f:
    st=f.read()
with open('/tmp/haproxy.cfg', 'w') as f:
    f.write(st.replace('172.17.0.4', httpd_ip))

In [ ]:

@hosts(haproxy_ip)
def setup_haproxy():
    run('yum -y -q install haproxy python ab')
    put ('/tmp/haproxy.cfg', '/etc/haproxy')
    #for port in ['1081', '1082', '1083']:
    # run("echo 'listen main{0}\n bind *:{0}\n server server1 {1}:{0}\n' >> /etc/haproxy/haproxy.cfg".format(port, httpd_ip))
    run("haproxy -f /etc/haproxy/haproxy.cfg")
execute(setup_haproxy)

In [ ]:
@hosts(httpd_ip)
def setup_httpd():
    run('yum -y -q install httpd python')
    for port in ['1081', '1082', '1083']:
     run("echo 'Listen {0}\n<VirtualHost *:{0}>\n CustomLog /tmp/{0}-access_log combined\n</VirtualHost>' >> /etc/httpd/conf/httpd.conf".format(port))
    run("httpd")
execute(setup_httpd)

In [ ]:
import requests
for port in ['1081', '1082', '1083']:
 print (requests.get('http://{0}:{1}'.format(haproxy_ip,port)))

In [ ]:
# get ansible facts
import os
print(os.popen('ANSIBLE_HOST_KEY_CHECKING=False ansible all -i {0} -e ansible_ssh_user=root -e ansible_ssh_pass=rootpass -m setup -t /tmp/ansible_facts'.format(haproxy_ip+','+httpd_ip)).read())

In [ ]:
@hosts(haproxy_ip)
def get_haproxy_cfg():
    #get('/etc/haproxy/haproxy.cfg', '/tmp')
    pass

applconn_ip='127.0.0.1'
@hosts(applconn_ip)
def setup_applconn():
    run('mkdir -p /tmp/ansible_facts')
    run('mkdir -p /var/tmp/rsyncgit/{0}/etc/haproxy'.format(haproxy_ip))

import sys
with open('/tmp/local_settings.py', 'w') as f:
 f.write('list_import_def=["import_ansible_facts", "import_haproxy", "import_testlogic"]')

@hosts(applconn_ip)
def upload_files_to_applconn():
    put('/tmp/haproxy.cfg', '/var/tmp/rsyncgit/{0}/etc/haproxy'.format(haproxy_ip))
    put('/tmp/ansible_facts/*', '/tmp/ansible_facts')
    #run ('ls -R /var/tmp/rsyncgit')
    #run(' > /var/tmp/applconn/local_settings.py')
    put('/tmp/local_settings.py', '/var/tmp/applconn/local_settings.py')
    run('cat /var/tmp/applconn/local_settings.py')
    run('/var/tmp/applconn/batchregister.py')
    
execute(get_haproxy_cfg)
execute(setup_applconn)
execute(upload_files_to_applconn)


Please open this 
http://myip:5000

After checking discovery is done and graph is successfully created, return back to the previous notebook